This is an agent that demonstrates the "Agent Reflection Pattern"

What is an Agent Reflection Pattern?
 
The Reflection Pattern is a powerful approach in AI, particularly for large language models (LLMs), where an iterative process of generation and self-assessment improves the output quality.

What's the use case that this code represents?

1. Get user input which is a problem statement to generate code (Use LLM1)
2. Generate V1 code
3. Reflect and refine (Use LLM2) - V1 code is reviewed, critiqued.  Returns reflection and refined V2 code
4. Output V2 code

In [2]:
# Install AI Suite
!pip3 install "aisuite[all]"


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip3 install --upgrade pip


In [3]:
# Custom pretty printing function
from pprint import pprint as pp
# Set a custom width for pretty-printing
def pprint(data, width=80):
    """Pretty print data with a specified width."""
    pp(data, width=width)# List of model identifiers to query

In [9]:
# Setting API keys - LLM1
import os
import aisuite as ai
from getpass import getpass
os.environ['ANTHROPIC_API_KEY'] = getpass('Enter your Anthropic API key: ')

Enter your Anthropic API key:  ········


In [15]:
# Setting API keys - LLM2
os.environ['OPENAI_API_KEY'] = getpass('Enter your OpenAI API key: ')

Enter your OpenAI API key:  ········


In [16]:
# Define a function to interact with the LLM Model

def generate_code(message, sys_message="You are a helpful agent.",
         model="openai:gpt-4o"):
    # Initialize the AI client for accessing the language model
    client = ai.Client()

    # Construct the messages list for the chat
    messages = [
        {"role": "system", "content": sys_message},
        {"role": "user", "content": message}
    ]

    # Send the messages to the model and get the response
    response = client.chat.completions.create(model=model, messages=messages)

    # Return the content of the model's response
    return response.choices[0].message.content

In [19]:
# Define the workflow

# Step 1 : Generate V1 code with LLM1
# Step 2 : Review and refine code with LLM2
# Step 3 : Generate V2 code with LLM2

def run_workflow(
    user_instructions: str,
    generation_model: str,
    reflection_model: str,   
):
    """
    End-to-end pipeline:
      1) Generate V1 code with LLM1
      2) Review and refine code with LLM2
      3) Generate V2 code with LLM2

    Returns a dict with all artifacts (codes, feedback).
    """
    # 1) Generate V1 code with LLM1
    code_v1 = generate_code(
        message=user_instructions,
        sys_message="You are a helpful coding agent",
        model=generation_model,
    )
    print('***** Original Code generated by generation_model *****')
    print(code_v1)

    # 2 and 3) Review and refine code with LLM2, Generate V2 code with LLM2
    code_v2 = generate_code(
        message=code_v1,
        sys_message="You are a helpful code review agent.  Review the code passed to you, improve and respond with a refined version of the code",
        model=reflection_model,
    )    

    print('***** Revised Code generated by reflection_model *****')
    print(code_v2)

    return {
        "code_v1": code_v1,
        "code_v1": code_v2,
    }

In [20]:
# Run the workflow

# Here, insert your updates
user_instructions="Create a program to find if a number is prime" # write your instruction here
generation_model="openai:gpt-4o-mini"
reflection_model="anthropic:claude-3-haiku-20240307"

# Run the complete agentic workflow
_ = run_workflow(
    user_instructions=user_instructions,
    generation_model=generation_model,
    reflection_model=reflection_model
)

***** Original Code generated by generation_model *****
Sure! Here’s a simple Python program to check if a number is prime:

```python
def is_prime(n):
    """Check if a number is prime."""
    if n <= 1:
        return False
    elif n <= 3:
        return True
    elif n % 2 == 0 or n % 3 == 0:
        return False
    
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    
    return True

# Input from the user
number = int(input("Enter a number to check if it's prime: "))

# Check if the number is prime
if is_prime(number):
    print(f"{number} is a prime number.")
else:
    print(f"{number} is not a prime number.")
```

### How the Program Works:
1. **Input**: The program prompts the user to enter a number.
2. **Prime Check**:
   - It first checks if the number is less than or equal to 1 (not prime) or is 2 or 3 (both prime).
   - Then it eliminates even numbers and multiples of 3.
   - For numbers greater than 3, it